# SETUP

In [1]:
import sys
sys.path.append("../src")

import torch
from data_loader import MaskDataLoader
from model import MaskClassifier
from trainer import Trainer
from predictor import Predictor
from submitter import Submitter

IMAGES_DIR      = "../data/images"
TRAIN_CSV       = "../data/train_labels.csv"
SAMPLE_SUB_CSV  = "../data/SampleSubmission.csv"
DEVICE          = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Device: cuda


# Load Data

In [2]:
loader = MaskDataLoader(
    images_dir=IMAGES_DIR,
    train_csv=TRAIN_CSV,
    sample_submission_csv=SAMPLE_SUB_CSV,
    val_split=0.15,
    batch_size=32
)
train_loader = loader.get_train_loader()
val_loader   = loader.get_val_loader()
test_loader  = loader.get_test_loader()
print(f"Train: {len(loader.train_df)} | Val: {len(loader.val_df)} | Test: {len(loader.test_df)}")

Train: 1111 | Val: 197 | Test: 509


# Primal Training EfficientNet
## Train

In [3]:
model   = MaskClassifier(dropout=0.4)
trainer = Trainer(model, DEVICE, lr=1e-4, patience=5)
trainer.fit(train_loader, val_loader, epochs=30)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\nilad/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:04<00:00, 4.39MB/s]


Epoch 001 | Train Loss: 0.6022 | Val Loss: 0.4071
 Saved best model (val_loss=0.4071)


Epoch 002 | Train Loss: 0.3116 | Val Loss: 0.1728
 Saved best model (val_loss=0.1728)


Epoch 003 | Train Loss: 0.1695 | Val Loss: 0.1341
 Saved best model (val_loss=0.1341)


...

Epoch 021 | Train Loss: 0.0219 | Val Loss: 0.0881


Epoch 022 | Train Loss: 0.0159 | Val Loss: 0.0993


Epoch 023 | Train Loss: 0.0212 | Val Loss: 0.1078


Epoch 024 | Train Loss: 0.0120 | Val Loss: 0.0813


Epoch 025 | Train Loss: 0.0135 | Val Loss: 0.0760
Early stopping at epoch 25.


## Predict & Submit

In [ ]:
predictor  = Predictor(model, DEVICE, model_path="models/best_model.pth")
fnames, probs = predictor.predict(test_loader)

submitter = Submitter(output_dir="../submissions")
submitter.save(fnames, probs, filename="submission_v1.csv")


## COMMIT

# git init
# git add .
# git commit -m "feat: baseline EfficientNet-B0 mask classifier with OOP pipeline [v0.1.0]"

Submission saved → ../submissions\submission_v1.csv


'../submissions\\submission_v1.csv'

# Finetuning EfficientNet
## Finetune with unfrozen layers

In [10]:
# Lower LR for fine-tuning — critical to avoid destroying pretrained weights
model_v2 = MaskClassifier(dropout=0.4)
model_v2.load_state_dict(torch.load("../models/best_model.pth"))  # start from best
model_v2.unfreeze_backbone(layers_from_end=3)

trainer_v2 = Trainer(model_v2, DEVICE, lr=3e-5, patience=5)  # 3x lower LR
trainer_v2.best_model_path = "../models/best_model_v2.pth"
trainer_v2.fit(train_loader, val_loader, epochs=20)

Unfroze last 3 backbone blocks.


Epoch 001 | Train Loss: 0.0198 | Val Loss: 0.0829
 Saved best model (val_loss=0.0829)


Epoch 002 | Train Loss: 0.0112 | Val Loss: 0.0808
 Saved best model (val_loss=0.0808)


Epoch 003 | Train Loss: 0.0202 | Val Loss: 0.0949


Epoch 004 | Train Loss: 0.0204 | Val Loss: 0.0829


...

Epoch 007 | Train Loss: 0.0050 | Val Loss: 0.0731


Epoch 008 | Train Loss: 0.0069 | Val Loss: 0.0776


Epoch 009 | Train Loss: 0.0055 | Val Loss: 0.0733


Epoch 010 | Train Loss: 0.0079 | Val Loss: 0.0810


Epoch 011 | Train Loss: 0.0076 | Val Loss: 0.0749
Early stopping at epoch 11.


## Predict with TTA & Submit

In [13]:
predictor_v2 = Predictor(model_v2, DEVICE, model_path="../models/best_model_v2.pth")
fnames, probs = predictor_v2.predict(test_loader, tta=True)
submitter = Submitter(output_dir="../submissions")
submitter.save(fnames, probs, filename="submission_v2.csv")

Submission saved → ../submissions\submission_v2.csv


'../submissions\\submission_v2.csv'

# Ensemble
## Train ResNet-50


In [3]:
from model import ResNetClassifier

model_resnet = ResNetClassifier(dropout=0.4)
trainer_resnet = Trainer(model_resnet, DEVICE, lr=1e-4, patience=5)
trainer_resnet.best_model_path = "../models/best_model_resnet.pth"
trainer_resnet.fit(train_loader, val_loader, epochs=30)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\nilad/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:21<00:00, 4.67MB/s]


Epoch 001 | Train Loss: 0.4824 | Val Loss: 0.0771
 Saved best model (val_loss=0.0771)


Epoch 002 | Train Loss: 0.1050 | Val Loss: 0.0489
 Saved best model (val_loss=0.0489)


Epoch 003 | Train Loss: 0.0508 | Val Loss: 0.0367
 Saved best model (val_loss=0.0367)


...

Epoch 006 | Train Loss: 0.0331 | Val Loss: 0.0439


Epoch 007 | Train Loss: 0.0171 | Val Loss: 0.0387


Epoch 008 | Train Loss: 0.0135 | Val Loss: 0.0341


Epoch 009 | Train Loss: 0.0144 | Val Loss: 0.0366


Epoch 010 | Train Loss: 0.0108 | Val Loss: 0.0360
Early stopping at epoch 10.


## Fine tune ResNet-50

In [4]:
model_resnet.unfreeze_backbone(layers_from_end=3)
trainer_resnet_ft = Trainer(model_resnet, DEVICE, lr=3e-5, patience=5)
trainer_resnet_ft.best_model_path = "../models/best_model_resnet_v2.pth"
trainer_resnet_ft.fit(train_loader, val_loader, epochs=20)

Unfroze last 3 ResNet layers.


Epoch 001 | Train Loss: 0.0117 | Val Loss: 0.0335
 Saved best model (val_loss=0.0335)


Epoch 002 | Train Loss: 0.0052 | Val Loss: 0.0323
 Saved best model (val_loss=0.0323)


Epoch 003 | Train Loss: 0.0043 | Val Loss: 0.0285
 Saved best model (val_loss=0.0285)


Epoch 004 | Train Loss: 0.0066 | Val Loss: 0.0251
 Saved best model (val_loss=0.0251)


...

Epoch 009 | Train Loss: 0.0040 | Val Loss: 0.0251


Epoch 010 | Train Loss: 0.0031 | Val Loss: 0.0277


Epoch 011 | Train Loss: 0.0017 | Val Loss: 0.0257


Epoch 012 | Train Loss: 0.0020 | Val Loss: 0.0254


Epoch 013 | Train Loss: 0.0033 | Val Loss: 0.0274
Early stopping at epoch 13.


## Ensemble predict & Submit

In [5]:
from predictor import EnsemblePredictor

pred_effnet = Predictor(MaskClassifier(), DEVICE, "../models/best_model_v2.pth")
pred_resnet = Predictor(ResNetClassifier(), DEVICE, "../models/best_model_resnet_v2.pth")

ensemble = EnsemblePredictor(
    predictors=[pred_effnet, pred_resnet],
    weights=[0.3, 0.7]  # equal trust — adjust after seeing val losses
)

fnames, probs = ensemble.predict(test_loader, tta=True)
submitter = Submitter(output_dir="../submissions")
submitter.save(fnames, probs, filename="submission_v3.csv")

Submission saved → ../submissions\submission_v3.csv


'../submissions\\submission_v3.csv'